In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol for the S&P 500 Index
ticker_symbol = "^GSPC"

# Fetch data for the S&P 500 Index
sp500_data = yf.download(ticker_symbol, start="2000-08-30", end="2017-01-01")

# Display the first few rows of the dataframe
# print(sp500_data.head())


[*********************100%%**********************]  1 of 1 completed


In [10]:
print(sp500_data.head())

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2000-08-30  1509.839966  1510.489990  1500.089966  1502.589966  1502.589966   
2000-08-31  1502.589966  1525.209961  1502.589966  1517.680054  1517.680054   
2000-09-01  1517.680054  1530.089966  1515.530029  1520.770020  1520.770020   
2000-09-05  1520.770020  1520.770020  1504.209961  1507.079956  1507.079956   
2000-09-06  1507.079956  1512.609985  1492.119995  1492.250000  1492.250000   

                Volume  
Date                    
2000-08-30   818400000  
2000-08-31  1056600000  
2000-09-01   767700000  
2000-09-05   838500000  
2000-09-06   995100000  


In [4]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol for Gold futures
ticker_symbol = "GC=F"

# Fetch data for Gold
gold_data = yf.download(ticker_symbol, start="2000-01-01", end="2017-01-01")

# Display the first few rows of the dataframe

[*********************100%%**********************]  1 of 1 completed


In [5]:
print(gold_data.head())

                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2000-08-30  273.899994  273.899994  273.899994  273.899994  273.899994       0
2000-08-31  274.799988  278.299988  274.799988  278.299988  278.299988       0
2000-09-01  277.000000  277.000000  277.000000  277.000000  277.000000       0
2000-09-05  275.799988  275.799988  275.799988  275.799988  275.799988       2
2000-09-06  274.200012  274.200012  274.200012  274.200012  274.200012       0


In [12]:
sp500_data.to_csv('/content/drive/My Drive/sp500.csv', index=False)
gold_data.to_csv('/content/drive/My Drive/gold.csv', index=False)

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import yfinance as yf

# Fetch S&P 500 data
ticker_symbol = "^GSPC"
sp500_data = yf.download(ticker_symbol, start="2015-01-01", end="2024-01-01")
sp500_close = sp500_data[['Close']]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(sp500_close)

# Create a training data set
train_data_len = int(np.ceil( len(scaled_data) * .95 ))
train_data = scaled_data[0:train_data_len, :]

# Split the data into x_train and y_train datasets
x_train, y_train = [], []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data for LSTM layer
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Prepare the test data (not shown here for brevity)
# Assume 'x_test' is prepared similarly and 'scaled_data' includes the entire dataset

# Predicting prices
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Plotting the results
train = sp500_close[:train_data_len]
valid = sp500_close[train_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()
